In [142]:
pd.options.display.max_colwidth = 1000

In [143]:
import numpy as np
import pandas as pd
import openai
import os
import string
import re
import random

Dataset taken from Kaggle (https://www.kaggle.com/datasets/mateibejan/multilingual-lyrics-for-genre-classification)

In [169]:
df_train = pd.read_csv("dataset/train.csv")

In [170]:
df_test = pd.read_csv("dataset/test.csv")

In [171]:
df_train.describe()

,Artist,Song,Genre,Language,Lyrics
count,290183,290182,290183,290179,290148
unique,11152,164357,10,33,249297
top,elvis presley,intro,Rock,en,Instrumental
freq,1611,163,121404,250197,540


In [172]:
df_train = df_train[df_train['Language']=='en']

In [173]:
df_train['Genre'].isnull().sum().sum()

0

In [174]:
df_train['Lyrics'].isnull().sum().sum()

0

In [175]:
df_train.describe()

,Artist,Song,Genre,Language,Lyrics
count,250197,250196,250197,250197,250197
unique,10701,137660,10,1,218551
top,elvis presley,home,Rock,en,"My heart is sad and lonely\nFor you I sigh, for you dear only\nWhy haven't you seen it\nI'm all for you body and soul\n\nI spend my days in longing\nAnd wondering why it's me you're wronging\nI tell you I mean it\nI'm all for you body and soul\n\nI can't believe it\nIt's hard to conceive it\nThat you'd turn away romance\nAre you pretending\nIt looks like the ending\nUnless I could have just one more chance to prove, dear\n\nMy life a wreck you're making\nYou know I'm yours for just the taking\nI'd gladly surrender myself to you body and soul\n\nMy life a wreck you're making\nYou know I'm yours for the very taking\nI'd gladly surrender myself to you body and soul"
freq,1598,148,107145,250197,89


In [176]:
df_train['Lyrics'].isnull().sum().sum()

0

Number of samples of each genre to be considered are as follows:

In [177]:
df_train['Genre'].unique()

array(['Rock', 'Metal', 'Pop', 'Indie', 'Folk', 'Electronic', 'R&B',
       'Jazz', 'Hip-Hop', 'Country'], dtype=object)

In [178]:
genre_list = ['Rock', 'Pop', 'Metal', 'Jazz', 'Folk', 'Indie', 'R&B', 'Hip-Hop', 'Electronic', 'Country']

Taking samples for different genres

In [179]:
rock_lyrics_train_sample_size = 3300
pop_lyrics_train_sample_size = 3100
metal_lyrics_train_sample_size = 2900
jazz_lyrics_train_sample_size = 2700
folk_lyrics_train_sample_size = 2500
indie_lyrics_train_sample_size = 2300
rnb_lyrics_train_sample_size = 2100
hip_hop_lyrics_train_sample_size = 1900
electronic_lyrics_train_sample_size = 1700
country_lyrics_train_sample_size = 1500

In [180]:
rock_lyrics_test_sample_size = 550
pop_lyrics_test_sample_size = 500
metal_lyrics_test_sample_size = 450
jazz_lyrics_test_sample_size = 400
folk_lyrics_test_sample_size = 350
indie_lyrics_test_sample_size = 300
rnb_lyrics_test_sample_size = 250
hip_hop_lyrics_test_sample_size = 200
electronic_lyrics_test_sample_size = 150
country_lyrics_test_sample_size = 100

In [181]:
genre_lyrics_train_sample_size_list = [rock_lyrics_train_sample_size, pop_lyrics_train_sample_size, metal_lyrics_train_sample_size,
                                jazz_lyrics_train_sample_size, folk_lyrics_train_sample_size, indie_lyrics_train_sample_size,
                                rnb_lyrics_train_sample_size, hip_hop_lyrics_train_sample_size, electronic_lyrics_train_sample_size,
                                country_lyrics_train_sample_size]

In [182]:
genre_samples_df_train_list = []

In [183]:
for i in range(len(genre_list)):
    genre_samples_df_train_list.append(df_train[df_train['Genre']==genre_list[i]].sample(n=genre_lyrics_train_sample_size_list[i]))

In [184]:
len(genre_samples_df_train_list)

10

Limiting a Lyrics to length of 512

In [186]:
lyrics_length = 512

Preprocessing Lyrics by removing punctuations, email address and digits

In [247]:
all_lyrics_in_all_sampled_genre = []
all_lyrics_in_all_sampled_genre_length = []
for i in genre_samples_df_train_list:
    total_lyrics_in_sampled_genre = len(i['Lyrics'])
    all_lyrics_in_sampled_genre = []
    all_lyrics_in_sampled_genre_length = []
    for lyrics_counter in range(total_lyrics_in_sampled_genre):
        lyrics = i['Lyrics'].iloc[[lyrics_counter]].values[0]
        #Removing punctuation
        lyrics = lyrics.translate(str.maketrans('', '', string.punctuation))
        #Removing email
        lyrics = re.sub("\S*@\S*\s?", "", lyrics)
        #Removing digits
        lyrics = re.sub("\d+", "", lyrics)
        #Trimming lyrics to length 512
        if len(lyrics) >= lyrics_length:
            lyrics = lyrics[0:lyrics_length]
        else:
            lyrics = lyrics.ljust(512, ' ')
        all_lyrics_in_sampled_genre.append(lyrics)
        all_lyrics_in_sampled_genre_length.append(len(lyrics))
    all_lyrics_in_all_sampled_genre.append(all_lyrics_in_sampled_genre)
    all_lyrics_in_all_sampled_genre_length.append(all_lyrics_in_sampled_genre_length)

In [188]:
for i in range(len(all_lyrics_in_all_sampled_genre)):
    genre_samples_df_train_list[i]['Lyrics'] = all_lyrics_in_all_sampled_genre[i]
    genre_samples_df_train_list[i] = genre_samples_df_train_list[i].drop_duplicates(subset = "Lyrics")

In [189]:
df_train = pd.concat(genre_samples_df_train_list, ignore_index=True)

In [190]:
df_train = df_train.sample(frac=1).reset_index(drop=True)

In [191]:
df_train.describe()

,Artist,Song,Genre,Language,Lyrics
count,23174,23174,23174,23174,23174
unique,5008,20185,10,1,23042
top,ella fitzgerald,home,Rock,en,I am not the only traveler\nWho has not repaid his debt\nIve been searching for a trail to follow again\nTake me back to the night we met\n\nAnd then I can tell myself\nWhat the hell Im supposed to do\nAnd then I can tell myself\nNot to ride along with you\n\nI had all and then most of you\nSome and now none of you\nTake me back to the night we met\nI dont know what Im supposed to do\nHaunted by the ghost of you\nOh take me back to the night we met\n\nWhen the night was full of terrors\nAnd your eyes were filled with tears\nW
freq,155,19,3290,23174,4


In [194]:
df_train['Genre'].value_counts()

Rock          3290
Pop           3077
Metal         2864
Folk          2470
Indie         2233
R&B           2088
Jazz          2062
Hip-Hop       1898
Electronic    1693
Country       1499
Name: Genre, dtype: int64

In [233]:
for i in range(10):
    print(len(df_train[df_train['Genre']==genre_list[i]]))

3290
3077
2864
2062
2470
2233
2088
1898
1693
1499


In [218]:
synthetic_rock_lyrics = []
synthetic_pop_lyrics = []
synthetic_metal_lyrics = []
synthetic_jazz_lyrics = []
synthetic_folk_lyrics = []
synthetic_indie_lyrics = []
synthetic_rnb_lyrics = []
synthetic_hip_hop_lyrics = []
synthetic_electronic_lyrics = []
synthetic_country_lyrics = []

In [219]:
synthetic_genre_lyrics_list = [synthetic_rock_lyrics, synthetic_pop_lyrics, synthetic_metal_lyrics, synthetic_jazz_lyrics,
                              synthetic_folk_lyrics, synthetic_indie_lyrics, synthetic_rnb_lyrics, synthetic_hip_hop_lyrics,
                              synthetic_electronic_lyrics, synthetic_country_lyrics]

In [220]:
synthetic_rock_lyrics_size = rock_lyrics_train_sample_size - rock_lyrics_train_sample_size
synthetic_pop_lyrics_size = rock_lyrics_train_sample_size - pop_lyrics_train_sample_size
synthetic_metal_lyrics_size = rock_lyrics_train_sample_size - metal_lyrics_train_sample_size
synthetic_jazz_lyrics_size = rock_lyrics_train_sample_size - jazz_lyrics_train_sample_size
synthetic_folk_lyrics_size = rock_lyrics_train_sample_size - folk_lyrics_train_sample_size
synthetic_indie_lyrics_size = rock_lyrics_train_sample_size - indie_lyrics_train_sample_size
synthetic_rnb_lyrics_size = rock_lyrics_train_sample_size - rnb_lyrics_train_sample_size
synthetic_hip_hop_lyrics_size = rock_lyrics_train_sample_size - hip_hop_lyrics_train_sample_size
synthetic_electronic_lyrics_size = rock_lyrics_train_sample_size - electronic_lyrics_train_sample_size
synthetic_country_lyrics_size = rock_lyrics_train_sample_size - country_lyrics_train_sample_size

In [221]:
synthetic_genre_lyrics_size_list = [synthetic_rock_lyrics_size, synthetic_pop_lyrics_size, synthetic_metal_lyrics_size,
                                   synthetic_jazz_lyrics_size, synthetic_folk_lyrics_size, synthetic_indie_lyrics_size,
                                   synthetic_rnb_lyrics_size, synthetic_hip_hop_lyrics_size, synthetic_electronic_lyrics_size,
                                   synthetic_country_lyrics_size]

As Lyrics Length is 512, number of tokens to be generated is limited to 100, 
assuming each token is approximately of 5 characters.

In [222]:
def synthetic_lyrics_generator(random_genre_lyrics):
    openai.api_key = ""

    prompt = "Generate a lyrics similar to this:" + random_genre_lyrics

    messages=[{"role": "user", "content": prompt}]

    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages,
      temperature=0.5,
      max_tokens=100,
      top_p=1.0,
      frequency_penalty=0.0,
      presence_penalty=0.0
    )

    return response['choices'][0]['message']['content']

In [2]:
for i in range(10):
    for j in range(len(df_train[df_train['Genre']==genre_list[i]])):
        random_genre_lyrics = df_train[df_train['Genre']==genre_list[i]]['Lyrics'].sample(n=1).to_string(index=False)
        generated_lyrics = synthetic_lyrics_generator(random_genre_lyrics)
        if generated_lyrics not in synthetic_genre_lyrics_list[i]:
            synthetic_genre_lyrics_list[i].append(generated_lyrics)

In [225]:
for i in range(len(synthetic_genre_lyrics_list)):
    print("---------Genre", i+1, "--------")
    for j in range(len(synthetic_genre_lyrics_list[i])):
        print("--------------Lyrics", j+1, "in", "Genre", i+1, "---------")
        print(synthetic_genre_lyrics_list[i][j])

---------Genre 1 --------
--------------Lyrics 1 in Genre 1 ---------
omanticizing
The weight of my broken family tree
My father, he carried burdens
With a mask of stoicism, he hid his plea

I've grown tired of wishful thinking
It weighs heavy on my chest
No room for second chances
I carry the weight as I leave the rest

Pride consumed my father's heart
But never did he take pride in me
I fought for my own survival
Yet I was painted as the villain, you see

You dev
---------Genre 2 --------
--------------Lyrics 1 in Genre 2 ---------
ding her with a stare\nShe plays the game without a care\nBut deep down, she's feeling lonely, oh yeah

She's a risk-taker, living on the edge
Leaving hearts shattered, with every pledge
But inside, there's a voice that begs
For someone to love her, to hold and protect

So why not take a chance
On this Hot Street, where passions dance
Feel the rhythm, let your heart advance
In this world of desire, take
---------Genre 3 --------
--------------Lyrics 1 in G